## ANALISA BENCHMARK PER JENIS ALAT BERAT (FILE PAK ABDI)

In [1]:
import pandas as pd
import numpy as np
import os
import re
import warnings
import gc

warnings.filterwarnings('ignore')

# ==============================================================================
# BAGIAN 1: LOAD MASTER DATA & GROUPING STRATEGY
# ==============================================================================
print("🚀 [1/3] MEMBACA & MEMBERSIHKAN MASTER DATA ALAT BERAT...")

def normalize_unit_name(name):
    if pd.isna(name): return ""
    name = str(name).upper().strip()
    name = re.sub(r'[/\-._]', ' ', name)
    name = name.replace('FORKLIF ', 'FORKLIFT ')
    if name.endswith('FORKLIF'): name = name + 'T'
    name = " ".join(name.split())
    return name

# 1. BACA FILE MASTER BARU
# Header ada di baris ke-2 (index 1) berdasarkan struktur file Anda
master_file = 'cost & bbm 2022 sd 2025.xlsx'

try:
    if os.path.exists(master_file):
        # Load Excel, skip row pertama, ambil header di row kedua
        df_map = pd.read_excel(master_file, header=1)
        
        # Rename kolom agar standar (Mapping manual berdasarkan posisi kolom di file Anda)
        # Asumsi urutan: NO, NAMA ALAT BERAT, DES 2025 (Lokasi), ALAT BERAT (Jenis), CAP
        # Kita rename kolomnya berdasarkan nama aslinya di Excel
        df_map.rename(columns={
            'NAMA ALAT BERAT': 'Unit_Original',
            'DES 2025': 'Lokasi',
            'ALAT BERAT': 'Jenis_Alat',
            'CAP': 'Capacity'
        }, inplace=True)
        
        # Filter Kolom Penting
        df_map = df_map[['Unit_Original', 'Lokasi', 'Jenis_Alat', 'Capacity']].copy()
        
        # 2. FILTER DATA (CLEANING)
        # Hapus baris yang Unit-nya kosong
        df_map.dropna(subset=['Unit_Original'], inplace=True)
        
        # Normalisasi Nama Unit
        df_map['Unit_Clean'] = df_map['Unit_Original'].apply(normalize_unit_name)
        
        # Filter 1: Hapus "DUMMY"
        df_map = df_map[~df_map['Unit_Clean'].str.contains('DUMMY', na=False)]
        
        # Filter 2: Hapus Capacity 0 (Konversi ke numeric dulu)
        df_map['Capacity'] = pd.to_numeric(df_map['Capacity'], errors='coerce').fillna(0)
        df_map = df_map[df_map['Capacity'] > 0]
        
        print(f"   -> Data Master Terbaca: {len(df_map)} unit valid (Non-Dummy, Cap > 0)")

        # 3. BUAT STRATEGI GROUPING (BENCHMARK GROUP)
        def determine_group(row):
            jenis = str(row['Jenis_Alat']).upper().strip()
            cap = row['Capacity']
            
            # LOGIKA GROUPING CUSTOM
            
            # Case A: Top Loader (Gabung 37, 40, 42 jadi satu)
            if 'TOP LOADER' in jenis:
                return 'TOP LOADER (37-42T)'
                
            # Case B: Crane (Pisah Small & Large)
            # Kapasitas yang ada: 41, 75, 80, 127
            elif 'CRANE' in jenis:
                if cap <= 45: # Mengcover Crane 41T
                    return 'CRANE SMALL (<= 45T)'
                else: # Mengcover 75, 80, 127
                    return 'CRANE LARGE (> 70T)'
            
            # Case C: Reach Stacker (Biasanya 45T)
            elif 'REACH STACKER' in jenis:
                return f"REACH STACKER ({int(cap)}T)"
                
            # Case D: Forklift (Grouping per range)
            elif 'FORKLIFT' in jenis:
                if cap <= 3: return 'FORKLIFT SMALL (<= 3T)'
                elif cap <= 5: return 'FORKLIFT MEDIUM (4-5T)'
                elif cap <= 10: return 'FORKLIFT LARGE (6-10T)'
                elif cap <= 16: return 'FORKLIFT X-LARGE (10-16T)'
                else: return f"FORKLIFT MONSTER ({int(cap)}T)"
            
            # Case E: Trailer / Tronton
            elif 'TRAILER' in jenis or 'TRONTON' in jenis:
                 return f"{jenis}" 

            # Default: Jenis + Cap
            else:
                return f"{jenis} ({int(cap)}T)"

        df_map['Benchmark_Group'] = df_map.apply(determine_group, axis=1)
        
        # 4. KONVERSI KE DICTIONARY UNTUK MAPPING CEPAT
        # Key: Nama Unit Bersih -> Value: {Group, Lokasi, Jenis, Cap}
        unit_master_dict = {}
        for _, row in df_map.iterrows():
            unit_master_dict[row['Unit_Clean']] = {
                'Benchmark_Group': row['Benchmark_Group'],
                'Lokasi': row['Lokasi'],
                'Jenis': row['Jenis_Alat'],
                'Capacity': row['Capacity']
            }
            
        print(f"✅ Mapping Selesai. Siap digunakan untuk {len(unit_master_dict)} unit.")
        print("\n--- Preview Grouping Baru ---")
        print(df_map['Benchmark_Group'].value_counts().head(10))
        
    else:
        print(f"❌ File '{master_file}' tidak ditemukan.")
        unit_master_dict = {}

except Exception as e:
    print(f"❌ Error membaca file Master: {e}")

🚀 [1/3] MEMBACA & MEMBERSIHKAN MASTER DATA ALAT BERAT...
   -> Data Master Terbaca: 241 unit valid (Non-Dummy, Cap > 0)
✅ Mapping Selesai. Siap digunakan untuk 241 unit.

--- Preview Grouping Baru ---
Benchmark_Group
TRAILER                   103
FORKLIFT MEDIUM (4-5T)     29
REACH STACKER (45T)        28
TRONTON                    21
FORKLIFT SMALL (<= 3T)     16
FORKLIFT LARGE (6-10T)     14
FORKLIFT MONSTER (32T)      6
TOP LOADER (37-42T)         5
REACH STACKER (42T)         5
CRANE LARGE (> 70T)         3
Name: count, dtype: int64


In [2]:
import gc

# ==============================================================================
# BAGIAN 2: BACA EXCEL (BBM AAB.xlsx) & HITUNG DELTA HARIAN
# ==============================================================================
print("\n🚀 [2/3] MEMPROSES FILE TRANSAKSI 'BBM AAB.xlsx'...")

filename_excel = 'BBM AAB.xlsx'
target_sheets = ['JAN', 'FEB', 'MAR', 'APR', 'MEI', 'JUN', 'JUL', 'AGT', 'SEP', 'OKT', 'NOV']
raw_data_list = []

if os.path.exists(filename_excel) and unit_master_dict:
    try:
        # Buka File Excel Sekali
        xls = pd.ExcelFile(filename_excel)
        print(f"   -> File ditemukan. Memproses Sheet JAN-NOV...")
        
        for sheet in target_sheets:
            if sheet in xls.sheet_names:
                # Baca Sheet tanpa header
                df = pd.read_excel(xls, sheet_name=sheet, header=None)
                
                # --- PARSING STRUKTUR ---
                unit_names = df.iloc[0].ffill() # Baris 0: Nama Unit
                headers = df.iloc[2]            # Baris 2: Metric (HM/LITER)
                dates = df.iloc[3:, 0]          # Kolom 0: Tanggal
                
                # Loop Kolom Data (Mulai kolom 1)
                for col in range(1, df.shape[1]):
                    header_str = str(headers[col]).strip().upper()
                    
                    # Filter Kolom: HM, LITER
                    if header_str in ['HM', 'LITER', 'KELUAR', 'PEMAKAIAN']:
                        metric_type = 'HM' if header_str == 'HM' else 'LITER'
                        
                        unit_raw = str(unit_names[col])
                        unit_clean = normalize_unit_name(unit_raw)
                        
                        # HANYA PROSES JIKA UNIT ADA DI MASTER MAP BARU
                        if unit_clean in unit_master_dict:
                            vals = pd.to_numeric(df.iloc[3:, col], errors='coerce')
                            
                            # Ambil Info dari Master Dictionary
                            info_unit = unit_master_dict[unit_clean]
                            
                            temp_df = pd.DataFrame({
                                'Date': dates,
                                'Month': sheet,
                                'Unit_Name': unit_clean,
                                'Benchmark_Group': info_unit['Benchmark_Group'], # Grouping Baru
                                'Lokasi': info_unit['Lokasi'],
                                'Jenis_Alat': info_unit['Jenis'],
                                'Capacity': info_unit['Capacity'],
                                'Metric': metric_type,
                                'Value': vals
                            })
                            
                            # Bersihkan data kosong
                            temp_df.dropna(subset=['Value', 'Date'], inplace=True)
                            if not temp_df.empty:
                                raw_data_list.append(temp_df)
                
                del df; gc.collect()
            else:
                print(f"      ⚠️ Sheet '{sheet}' tidak ditemukan.")
                
    except Exception as e:
        print(f"   ❌ Error membaca Excel Transaksi: {e}")
else:
    if not unit_master_dict:
        print("   ❌ Master Data kosong, proses dihentikan.")
    else:
        print(f"   ❌ File '{filename_excel}' TIDAK DITEMUKAN.")

# --- PROSES PERHITUNGAN (PIVOT & DELTA) ---
unit_perf_data = []

if raw_data_list:
    print("   -> Menggabungkan Data & Pivot...")
    df_all = pd.concat(raw_data_list, ignore_index=True)
    
    # Konversi Tanggal
    df_all['Date'] = pd.to_datetime(df_all['Date'], dayfirst=True, errors='coerce')
    df_all.dropna(subset=['Date'], inplace=True)
    
    # PIVOT TABLE
    # Index harus menyertakan properti unit agar tidak hilang saat unstack
    df_pivot = df_all.pivot_table(
        index=['Unit_Name', 'Benchmark_Group', 'Lokasi', 'Jenis_Alat', 'Capacity', 'Date'],
        columns='Metric',
        values='Value',
        aggfunc='sum'
    ).reset_index()
    
    if 'HM' not in df_pivot.columns: df_pivot['HM'] = 0
    if 'LITER' not in df_pivot.columns: df_pivot['LITER'] = 0
    
    # Sortir Kronologis
    df_pivot.sort_values(by=['Unit_Name', 'Date'], inplace=True)
    
    print("   -> Menghitung Delta HM Harian...")
    # Hitung Delta HM
    df_pivot['Delta_HM'] = df_pivot.groupby('Unit_Name')['HM'].diff()
    
    # Cleaning Delta
    df_pivot.loc[df_pivot['Delta_HM'] < 0, 'Delta_HM'] = 0 # Reset/Error
    df_pivot['Delta_HM'] = df_pivot['Delta_HM'].fillna(0)  # Hari pertama
    
    # AGREGASI TOTAL PER UNIT (JAN-NOV)
    final_stats = df_pivot.groupby(['Unit_Name', 'Benchmark_Group', 'Lokasi', 'Jenis_Alat', 'Capacity']).agg({
        'LITER': 'sum',
        'Delta_HM': 'sum'
    }).reset_index()
    
    final_stats.rename(columns={'LITER': 'Total_Liter', 'Delta_HM': 'Total_HM_Work'}, inplace=True)
    
    # Konversi ke Dictionary
    unit_perf_data = final_stats.to_dict('records')
        
    print(f"✅ Selesai. Data HM & BBM dihitung.")
    print(f"   Total Unit Terproses: {len(unit_perf_data)}")
    
else:
    print("❌ Tidak ada data transaksi yang berhasil diekstrak.")


🚀 [2/3] MEMPROSES FILE TRANSAKSI 'BBM AAB.xlsx'...
   -> File ditemukan. Memproses Sheet JAN-NOV...
   -> Menggabungkan Data & Pivot...
   -> Menghitung Delta HM Harian...
✅ Selesai. Data HM & BBM dihitung.
   Total Unit Terproses: 193


In [8]:
# ==============================================================================
# BAGIAN 3: ANALISA BENCHMARK & EXPORT EXCEL (REVISI: PISAH INAKTIF)
# ==============================================================================
print("🚀 [3/3] MENGHITUNG BENCHMARK & MEMBUAT LAPORAN...")

if unit_perf_data:
    df_raw = pd.DataFrame(unit_perf_data)
    
    # 1. HITUNG FUEL RATIO INDIVIDU
    def calc_ratio(row):
        if row['Total_HM_Work'] > 0:
            return row['Total_Liter'] / row['Total_HM_Work']
        return 0
    
    df_raw['Fuel_Ratio'] = df_raw.apply(calc_ratio, axis=1)
    
    # 2. HITUNG MEDIAN PER 'BENCHMARK GROUP'
    # Hanya unit yang aktif (HM > 0 & BBM > 0) yang ikut menyumbang nilai benchmark
    df_valid_for_bench = df_raw[
        (df_raw['Total_HM_Work'] > 0) & 
        (df_raw['Total_Liter'] > 0)
    ].copy()
    
    # Hitung Median per Group
    benchmark_stats = df_valid_for_bench.groupby('Benchmark_Group')['Fuel_Ratio'].median().reset_index()
    benchmark_stats.rename(columns={'Fuel_Ratio': 'Group_Benchmark_Median'}, inplace=True)
    
    # Gabungkan kembali ke data utama
    df_final = pd.merge(df_raw, benchmark_stats, on='Benchmark_Group', how='left')
    
    # 3. TENTUKAN STATUS (EFISIEN / BOROS / INAKTIF)
    def get_status(row):
        # Jika data 0/Kosong
        if row['Total_HM_Work'] <= 0 or row['Total_Liter'] <= 0: 
            return "DATA TIDAK LENGKAP / INAKTIF"
        
        bench = row['Group_Benchmark_Median']
        if pd.isna(bench): return "SINGLE UNIT (NO COMPARISON)"
        
        if row['Fuel_Ratio'] <= bench: 
            return "EFISIEN"
        else: 
            return "BOROS"

    df_final['Performance_Status'] = df_final.apply(get_status, axis=1)
    
    # Hitung Pemborosan Liter (Hanya untuk yang statusnya BOROS, otomatis Inaktif tidak terhitung)
    df_final['Potensi_Pemborosan_Liter'] = 0
    boros_mask = df_final['Performance_Status'] == "BOROS"
    df_final.loc[boros_mask, 'Potensi_Pemborosan_Liter'] = (
        (df_final.loc[boros_mask, 'Fuel_Ratio'] - df_final.loc[boros_mask, 'Group_Benchmark_Median']) 
        * df_final.loc[boros_mask, 'Total_HM_Work']
    )
    
    # Formatting Rapi
    df_final['Fuel_Ratio'] = df_final['Fuel_Ratio'].round(2)
    df_final['Group_Benchmark_Median'] = df_final['Group_Benchmark_Median'].round(2)
    df_final['Potensi_Pemborosan_Liter'] = df_final['Potensi_Pemborosan_Liter'].round(0)
    
    # Sortir
    df_final.sort_values(['Benchmark_Group', 'Fuel_Ratio'], inplace=True)
    
    # --- PISAHKAN DATA AKTIF & INAKTIF ---
    # Unit Aktif: Status BUKAN "DATA TIDAK LENGKAP / INAKTIF"
    df_active = df_final[df_final['Performance_Status'] != "DATA TIDAK LENGKAP / INAKTIF"].copy()
    
    # Unit Inaktif: Status ADALAH "DATA TIDAK LENGKAP / INAKTIF"
    df_inactive = df_final[df_final['Performance_Status'] == "DATA TIDAK LENGKAP / INAKTIF"].copy()
    
    # 4. EXPORT KE EXCEL
    output_file = 'Benchmark_Per_Alat_Berat_Data_Baru.xlsx'
    try:
        with pd.ExcelWriter(output_file) as writer:
            
            # --- SHEET 1: RAPOR UNIT AKTIF (Data Bersih) ---
            cols = [
                'Benchmark_Group', 'Jenis_Alat', 'Capacity', 'Lokasi', 'Unit_Name', 
                'Total_Liter', 'Total_HM_Work', 'Fuel_Ratio', 
                'Group_Benchmark_Median', 'Performance_Status', 'Potensi_Pemborosan_Liter'
            ]
            cols_avail = [c for c in cols if c in df_active.columns]
            df_active[cols_avail].to_excel(writer, sheet_name='Rapor_Unit_Aktif', index=False)
            
            # --- SHEET 2: SUMMARY GROUP (Hanya dari Unit Aktif) ---
            # Grouping hanya berdasarkan df_active agar angka populasi & rata-rata tidak bias
            summary_grp = df_active.groupby('Benchmark_Group').agg({
                'Unit_Name': 'count',
                'Total_Liter': 'sum',
                'Total_HM_Work': 'sum',
                'Group_Benchmark_Median': 'first',
                'Potensi_Pemborosan_Liter': 'sum'
            }).reset_index().rename(columns={'Unit_Name': 'Populasi_Unit_Aktif'})
            
            summary_grp.to_excel(writer, sheet_name='Summary_Group', index=False)
            
            # --- SHEET 3: UNIT INAKTIF (Untuk Pengecekan) ---
            # Kolom BBM & HM ditampilkan agar tahu mana yang 0
            cols_inactive = ['Benchmark_Group', 'Jenis_Alat', 'Lokasi', 'Unit_Name', 'Total_Liter', 'Total_HM_Work']
            df_inactive[cols_inactive].to_excel(writer, sheet_name='Unit_Inaktif', index=False)
            
        print(f"\n💾  SUKSES! Laporan Tersimpan: '{output_file}'")
        print(f"    - Unit Aktif: {len(df_active)}")
        print(f"    - Unit Inaktif (Dipisah): {len(df_inactive)}")
        
        # Preview
        print("\n--- Preview Summary (Hanya Unit Aktif) ---")
        print(summary_grp[['Benchmark_Group', 'Populasi_Unit_Aktif', 'Group_Benchmark_Median']].head(10).to_string(index=False))
        
    except Exception as e:
        print(f"⚠️ Gagal save Excel: {e}")
else:
    print("❌ Tidak ada data untuk dianalisa.")

🚀 [3/3] MENGHITUNG BENCHMARK & MEMBUAT LAPORAN...

💾  SUKSES! Laporan Tersimpan: 'Benchmark_Per_Alat_Berat_Data_Baru.xlsx'
    - Unit Aktif: 148
    - Unit Inaktif (Dipisah): 45

--- Preview Summary (Hanya Unit Aktif) ---
       Benchmark_Group  Populasi_Unit_Aktif  Group_Benchmark_Median
   CRANE LARGE (> 70T)                    2                    6.62
  CRANE SMALL (<= 45T)                    3                    5.86
FORKLIFT LARGE (6-10T)                   13                    4.69
FORKLIFT MEDIUM (4-5T)                   27                    2.90
FORKLIFT MONSTER (28T)                    2                   11.65
FORKLIFT MONSTER (32T)                    5                   10.29
FORKLIFT SMALL (<= 3T)                   13                    1.83
   REACH STACKER (42T)                    5                   12.56
   REACH STACKER (45T)                   27                   15.07
      SIDE LOADER (5T)                    2                    3.76


## ANALISA BENCHMARK PER JENIS ALAT BERAT (FILE CABANG)

In [3]:
import pandas as pd
import numpy as np
import os
import re
import gc
import warnings

warnings.filterwarnings('ignore')

# ==============================================================================
# BAGIAN 1: SETUP & LOAD MASTER MAPPING
# ==============================================================================
print("🚀 [1/3] MEMBACA MASTER DATA JENIS ALAT...")

def normalize_unit_name(name):
    if pd.isna(name): return ""
    name = str(name).upper().strip()
    name = re.sub(r'[/\-._]', ' ', name)
    name = name.replace('FORKLIF ', 'FORKLIFT ')
    if name.endswith('FORKLIF'): name = name + 'T'
    name = " ".join(name.split())
    return name

mapping_file = 'Master_Mapping_Jenis_Alat.xlsx'
unit_type_map = {}

# Cek keberadaan file mapping (Excel atau CSV)
if os.path.exists(mapping_file):
    df_map = pd.read_excel(mapping_file)
elif os.path.exists(mapping_file.replace('.xlsx', '.csv')):
    df_map = pd.read_csv(mapping_file.replace('.xlsx', '.csv'))
else:
    df_map = pd.DataFrame()

if not df_map.empty:
    for _, row in df_map.iterrows():
        # Ambil kolom Unit dan Jenis (sesuaikan index kolom jika perlu)
        unit = str(row.get('Unit_Name_BBM', row.iloc[0])) 
        jenis = str(row.get('Jenis_Alat', row.iloc[2]))
        
        if jenis.upper() != 'UNKNOWN':
            unit_type_map[normalize_unit_name(unit)] = jenis
            
    print(f"✅ Master Data Terbaca. Total Unit Terdaftar: {len(unit_type_map)}")
else:
    print(f"❌ File Mapping '{mapping_file}' tidak ditemukan! Kategori alat mungkin kosong.")

🚀 [1/3] MEMBACA MASTER DATA JENIS ALAT...
✅ Master Data Terbaca. Total Unit Terdaftar: 242


In [8]:
# ==============================================================================
# BAGIAN 2: BACA EXCEL (XLSX) & HITUNG DELTA HARIAN (REVISI LOGIC)
# ==============================================================================
print("🚀 [2/3] MEMPROSES FILE EXCEL 'BBM AAB.xlsx'...")

filename_excel = 'BBM AAB.xlsx'
target_sheets = ['JAN', 'FEB', 'MAR', 'APR', 'MEI', 'JUN', 'JUL', 'AGT', 'SEP', 'OKT', 'NOV']
raw_data_list = []

if os.path.exists(filename_excel):
    try:
        # Buka File Excel Sekali
        xls = pd.ExcelFile(filename_excel)
        print(f"   -> File ditemukan. Sheet tersedia: {len(xls.sheet_names)}")
        
        for sheet in target_sheets:
            if sheet in xls.sheet_names:
                print(f"   -> Membaca Sheet: {sheet}")
                
                # Baca Sheet tanpa header (Baris 0=Unit, Baris 2=Metric)
                df = pd.read_excel(xls, sheet_name=sheet, header=None)
                
                # --- PARSING STRUKTUR (SAMA SEPERTI NOTEBOOK) ---
                unit_names = df.iloc[0].ffill() # Baris 0: Nama Unit
                headers = df.iloc[2]            # Baris 2: Metric (HM/LITER)
                dates = df.iloc[3:, 0]          # Kolom 0: Tanggal
                
                # Loop Kolom Data (Mulai kolom 1)
                for col in range(1, df.shape[1]):
                    header_str = str(headers[col]).strip().upper()
                    
                    # Filter Kolom: HM, LITER, KELUAR
                    if header_str in ['HM', 'LITER', 'KELUAR', 'PEMAKAIAN']:
                        metric_type = 'HM' if header_str == 'HM' else 'LITER'
                        
                        unit_raw = str(unit_names[col])
                        unit_clean = normalize_unit_name(unit_raw)
                        
                        # Hanya proses jika unit ada di Master Mapping
                        if unit_clean in unit_type_map:
                            vals = pd.to_numeric(df.iloc[3:, col], errors='coerce')
                            
                            temp_df = pd.DataFrame({
                                'Date': dates,
                                'Month': sheet, # Info bulan
                                'Unit_Name': unit_clean,
                                'Metric': metric_type,
                                'Value': vals
                            })
                            
                            temp_df.dropna(subset=['Value', 'Date'], inplace=True)
                            if not temp_df.empty:
                                raw_data_list.append(temp_df)
                
                del df; gc.collect()
            else:
                print(f"      ⚠️ Sheet '{sheet}' tidak ditemukan.")
                
    except Exception as e:
        print(f"   ❌ Error membaca Excel: {e}")
else:
    print(f"   ❌ File '{filename_excel}' TIDAK DITEMUKAN di folder ini.")

# --- PROSES PERHITUNGAN (LOGIKA REVISI: TANPA FILTER 24 JAM) ---
if raw_data_list:
    print("   -> Menggabungkan Data & Pivot...")
    df_all = pd.concat(raw_data_list, ignore_index=True)
    
    # Konversi Tanggal
    df_all['Date'] = pd.to_datetime(df_all['Date'], dayfirst=True, errors='coerce')
    df_all.dropna(subset=['Date'], inplace=True)
    
    # 1. PIVOT (Baris=Unit+Tgl, Kolom=HM,LITER)
    df_pivot = df_all.pivot_table(
        index=['Unit_Name', 'Date'],
        columns='Metric',
        values='Value',
        aggfunc='sum'
    ).reset_index()
    
    if 'HM' not in df_pivot.columns: df_pivot['HM'] = 0
    if 'LITER' not in df_pivot.columns: df_pivot['LITER'] = 0
    
    # Sortir Kronologis (Wajib buat Delta)
    df_pivot.sort_values(by=['Unit_Name', 'Date'], inplace=True)
    
    print("   -> Menghitung Delta HM Harian...")
    # 2. HITUNG DELTA HM (Hari Ini - Kemarin)
    df_pivot['Delta_HM'] = df_pivot.groupby('Unit_Name')['HM'].diff()
    
    # Cleaning Delta:
    df_pivot.loc[df_pivot['Delta_HM'] < 0, 'Delta_HM'] = 0 # Reset/Error jadi 0
    df_pivot['Delta_HM'] = df_pivot['Delta_HM'].fillna(0)  # Hari pertama jadi 0
    
    # --- PERBAIKAN DI SINI ---
    # Filter > 24 jam DIHAPUS agar akumulasi jam kerja (seperti LANDAK & BOSS 3) tetap terhitung.
    # df_pivot.loc[df_pivot['Delta_HM'] > 24, 'Delta_HM'] = 0 
    
    # 3. AGREGASI TOTAL (JAN-NOV)
    final_stats = df_pivot.groupby('Unit_Name').agg({
        'LITER': 'sum',
        'Delta_HM': 'sum'
    }).reset_index()
    
    final_stats.rename(columns={'LITER': 'Total_Liter', 'Delta_HM': 'Total_HM_Work'}, inplace=True)
    
    # Filter Output
    unit_perf_data = final_stats[
        (final_stats['Total_Liter'] > 0) | (final_stats['Total_HM_Work'] > 0)
    ].to_dict('records')
    
    for rec in unit_perf_data:
        rec['Jenis_Alat'] = unit_type_map.get(rec['Unit_Name'], 'UNKNOWN')
        
    print(f"✅ Selesai. Data HM & BBM dihitung dari XLSX Sheet JAN-NOV.")
    print(f"   Total Unit Terproses: {len(unit_perf_data)}")
    
else:
    print("❌ Tidak ada data yang berhasil diekstrak.")
    unit_perf_data = []

🚀 [2/3] MEMPROSES FILE EXCEL 'BBM AAB.xlsx'...
   -> File ditemukan. Sheet tersedia: 11
   -> Membaca Sheet: JAN
   -> Membaca Sheet: FEB
   -> Membaca Sheet: MAR
   -> Membaca Sheet: APR
   -> Membaca Sheet: MEI
   -> Membaca Sheet: JUN
   -> Membaca Sheet: JUL
   -> Membaca Sheet: AGT
   -> Membaca Sheet: SEP
   -> Membaca Sheet: OKT
   -> Membaca Sheet: NOV
   -> Menggabungkan Data & Pivot...
   -> Menghitung Delta HM Harian...
✅ Selesai. Data HM & BBM dihitung dari XLSX Sheet JAN-NOV.
   Total Unit Terproses: 240


In [7]:
# ==============================================================================
# BAGIAN 3: ANALISA BENCHMARK & EXPORT
# ==============================================================================
print("🚀 [3/3] MEMBUAT LAPORAN EXCEL...")

if unit_perf_data:
    df_raw = pd.DataFrame(unit_perf_data)
    
    # 1. Hitung Ratio
    df_raw['Fuel_Ratio'] = df_raw.apply(
        lambda x: x['Total_Liter'] / x['Total_HM_Work'] if x['Total_HM_Work'] > 0 else 0, axis=1
    )
    
    # 2. Benchmark (Median)
    df_valid = df_raw[(df_raw['Total_HM_Work'] > 0) & (df_raw['Total_Liter'] > 0)].copy()
    median_stats = df_valid.groupby('Jenis_Alat')['Fuel_Ratio'].median().reset_index(name='Group_Benchmark_Ratio')
    
    df_final = pd.merge(df_raw, median_stats, on='Jenis_Alat', how='left')
    
    # 3. Status
    def get_status(row):
        if row['Total_HM_Work'] == 0 or row['Total_Liter'] == 0:
            return "DATA TIDAK LENGKAP"
        
        bench = row['Group_Benchmark_Ratio']
        if pd.isna(bench): return "SINGLE UNIT"
        
        if row['Fuel_Ratio'] <= bench: return "Efisien"
        else: return "Boros"

    df_final['Performance_Status'] = df_final.apply(get_status, axis=1)
    
    # Formatting
    df_final['Fuel_Ratio'] = df_final['Fuel_Ratio'].round(2)
    df_final['Group_Benchmark_Ratio'] = df_final['Group_Benchmark_Ratio'].round(2)
    df_final.sort_values(['Jenis_Alat', 'Fuel_Ratio'], inplace=True)
    
    # 4. Summary Table
    summary_list = []
    for jenis, group in df_final.groupby('Jenis_Alat'):
        valid_grp = group[group['Performance_Status'].isin(["Efisien", "Boros"])]
        populasi = len(valid_grp)
        
        if populasi > 0:
            best = valid_grp.iloc[0]
            worst = valid_grp.iloc[-1]
            summary_list.append({
                'Jenis_Alat': jenis,
                'Populasi_Valid': populasi,
                'Benchmark_Median': best['Group_Benchmark_Ratio'],
                'Best_Unit': best['Unit_Name'],
                'Best_Ratio': best['Fuel_Ratio'],
                'Worst_Unit': worst['Unit_Name'] if populasi > 1 else "-",
                'Worst_Ratio': worst['Fuel_Ratio'] if populasi > 1 else "-"
            })
            
    df_summary = pd.DataFrame(summary_list)
    
    # 5. Save
    output_file = 'Analisa_Benchmark_Per_Alat_Berat.xlsx'
    try:
        with pd.ExcelWriter(output_file) as writer:
            df_summary.to_excel(writer, sheet_name='Summary_Per_Jenis', index=False)
            
            cols = ['Jenis_Alat', 'Unit_Name', 'Fuel_Ratio', 'Group_Benchmark_Ratio', 'Performance_Status', 'Total_Liter', 'Total_HM_Work']
            df_final[cols].to_excel(writer, sheet_name='Rapor_Per_Unit', index=False)
            
        print(f"\n💾  SUKSES! Laporan tersimpan: '{output_file}'")
        print("\n--- PREVIEW SUMMARY ---")
        print(df_summary.head(10).to_string(index=False))
        
    except Exception as e:
        print(f"⚠️ Gagal save Excel: {e}")
else:
    print("❌ Tidak ada data.")

🚀 [3/3] MEMBUAT LAPORAN EXCEL...

💾  SUKSES! Laporan tersimpan: 'Analisa_Benchmark_Per_Alat_Berat.xlsx'

--- PREVIEW SUMMARY ---
   Jenis_Alat  Populasi_Valid  Benchmark_Median                   Best_Unit  Best_Ratio                 Worst_Unit  Worst_Ratio
        CRANE               5              6.03                        RG01        5.79 CRANE LB 80T LUCKY DOLPHIN         6.67
     FORKLIFT              60              3.10                       PALEM        0.67                    MAGNETO        11.77
REACH STACKER              32             14.77          KALAMR5 KINGKONG 1        9.12                  KALMAR 20        20.94
  SIDE LOADER               5              7.69          SIDE LOADER BOSS 3        0.64         SIDE LOUDER KALMAR         8.43
   TOP LOADER               4             11.63 TOP LOADER MITS CENDRAWASIH        0.21  TOP LOADER MITS 42T BADAK        22.32
      TRAILER              34              4.49                   L 9049 US        2.51                

In [14]:
# ==============================================================================
# DIAGNOSA: AUDIT UNIT YANG HILANG DARI ANALISA
# ==============================================================================
print("🚀 MENJALANKAN DIAGNOSA UNIT HILANG...")

# 1. Ambil List Semua Unit Target (Dari Master Mapping)
if 'unit_type_map' not in locals():
    # Reload jika perlu
    mapping_file = 'Master_Mapping_Jenis_Alat.xlsx'
    df_map = pd.read_excel(mapping_file)
    target_units = set(df_map[df_map['Jenis_Alat'] != 'UNKNOWN']['Unit_Name_BBM'].unique())
else:
    target_units = set(unit_type_map.keys())

print(f"1️⃣  Total Unit Target (Punya Jenis Alat): {len(target_units)} unit")

# 2. Ambil List Unit yang Lolos Analisa (Dari df_final Block 3)
if 'df_final' in locals():
    passed_units = set(df_final['Unit_Name'].unique())
else:
    passed_units = set()

print(f"2️⃣  Total Unit Lolos Analisa: {len(passed_units)} unit")

# 3. Cari Selisih (Unit yang Hilang)
missing_units = target_units - passed_units
print(f"3️⃣  Total Unit Terbuang/Hilang: {len(missing_units)} unit")

# 4. Investigasi Kenapa Hilang (Cek Data Mentah di unit_perf_data)
# Kita convert list unit_perf_data ke DataFrame dulu untuk cek
if 'unit_perf_data' in locals() and unit_perf_data:
    df_raw_check = pd.DataFrame(unit_perf_data)
    
    # Agregasi per unit (karena di raw data bisa banyak baris per unit)
    df_check_agg = df_raw_check.groupby('Unit_Name').agg({
        'Total_Liter': 'sum',
        'Total_HM_Work': 'sum'
    }).reset_index()
    
    report_missing = []
    
    for unit in missing_units:
        # Cek apakah unit ini ada di data mentah (BBM)?
        record = df_check_agg[df_check_agg['Unit_Name'] == unit]
        
        if record.empty:
            reason = "TIDAK ADA TRANSAKSI (Kosong di BBM)"
            detil = "-"
        else:
            liters = record.iloc[0]['Total_Liter']
            hm = record.iloc[0]['Total_HM_Work']
            
            if hm <= 0 and liters <= 0:
                reason = "DATA NULL (HM 0 & BBM 0)"
                detil = f"HM={hm}, BBM={liters}"
            elif hm <= 0:
                reason = "HM ERROR (HM tidak bergerak/0)"
                detil = f"HM={hm}, BBM={liters}"
            elif liters <= 0:
                reason = "TIDAK ADA BBM (BBM 0)"
                detil = f"HM={hm}, BBM={liters}"
            else:
                reason = "UNKNOWN ERROR"
                detil = f"HM={hm}, BBM={liters}"
        
        # Ambil Jenis Alat
        jenis = unit_type_map.get(unit, "UNKNOWN")
        
        report_missing.append({
            'Unit_Name': unit,
            'Jenis_Alat': jenis,
            'Alasan_Dibuang': reason,
            'Detail_Angka': detil
        })
        
    # 5. Export Laporan Missing
    if report_missing:
        df_missing_report = pd.DataFrame(report_missing)
        
        # Sortir biar rapi
        df_missing_report.sort_values(['Alasan_Dibuang', 'Unit_Name'], inplace=True)
        
        output_file_missing = 'Unit_Exceptional_Analisa_Per_Alat_Berat.xlsx'
        df_missing_report.to_excel(output_file_missing, index=False)
        
        print(f"\n⚠️  DITEMUKAN {len(df_missing_report)} UNIT BERMASALAH.")
        print(f"    File Laporan: '{output_file_missing}'")
        print("\n--- CONTOH 10 UNIT YANG DIBUANG ---")
        print(df_missing_report[['Unit_Name', 'Alasan_Dibuang', 'Detail_Angka']].head(10).to_string(index=False))
    else:
        print("✅ Aneh, tidak ada report missing padahal ada selisih angka.")

else:
    print("❌ Data mentah 'unit_perf_data' tidak ditemukan. Jalankan Block 2 dulu.")

🚀 MENJALANKAN DIAGNOSA UNIT HILANG...
1️⃣  Total Unit Target (Punya Jenis Alat): 242 unit
2️⃣  Total Unit Lolos Analisa: 191 unit
3️⃣  Total Unit Terbuang/Hilang: 51 unit

⚠️  DITEMUKAN 51 UNIT BERMASALAH.
    File Laporan: 'Unit_Exceptional_Analisa_Per_Alat_Berat.xlsx'

--- CONTOH 10 UNIT YANG DIBUANG ---
               Unit_Name                 Alasan_Dibuang       Detail_Angka
               L 9743 UR HM ERROR (HM tidak bergerak/0)  HM=0.0, BBM=170.0
                    SOKA HM ERROR (HM tidak bergerak/0)  HM=0.0, BBM=55.58
B 9137 MJ (EX B 9494 JA)          TIDAK ADA BBM (BBM 0) HM=4616.0, BBM=0.0
               B 9157 ZJ          TIDAK ADA BBM (BBM 0) HM=4375.0, BBM=0.0
               B 9220 BL          TIDAK ADA BBM (BBM 0) HM=4134.0, BBM=0.0
               B 9224 BL          TIDAK ADA BBM (BBM 0) HM=3782.0, BBM=0.0
               B 9493 JA          TIDAK ADA BBM (BBM 0) HM=4311.0, BBM=0.0
               B 9566 ZB          TIDAK ADA BBM (BBM 0) HM=4399.0, BBM=0.0
               B 